In [33]:
import openai
from dotenv import load_dotenv
from Chatbot import Chatbot
import os
from langchain.document_loaders import DirectoryLoader,PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import openai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI


In [26]:
pdf= "article.pdf"

In [27]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [28]:
chatbot = Chatbot()

loader  = PyPDFDirectoryLoader("directory/")


In [29]:
docs = loader.load()

In [30]:
len(docs)

18

In [24]:
chunks[0]

Document(page_content='Article\nPhasing out the blast furnace to meet global\nclimate targets\nIron and steel production accounts for 7% of anthropogenic greenhouse gas\nemissions. We estimate expected future emissions from steel based on animproved approach to committed emissions accounting that uses actualequipment-level data. Without starting to phase out unmitigated blast furnaces\nsoon the operation of current steel production equipment will consume signiﬁcant\namounts of the remaining 1.5\n/C14C carbon budget. The age of emission-intensive\nassets should be better monitored, and research should explore regulatoryphaseouts of such assets.\nValentin Vogl, Olle Olsson,\nBjo¨rn Nykvist\nvalentin.vogl@miljo.lth.se\nHighlights\nFirst estimation of committed\nemissions based on actual\nindustry equipment data\nThe median historic blast furnace\ncampaign length is 17 years\nCO 2emissions of 21 Gt to be\nexpected for immediate blastfurnace phase-out case\n10 years of inaction and steel\nc

In [34]:
embeddings = OpenAIEmbeddings()
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

In [35]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000 ,chunk_overlap=100)
chunks = splitter.split_documents(docs)
vectorestore = Chroma.from_documents(chunks, embeddings)
#chain = chatbot.build_chain(vectorstore)

In [37]:
chain = ConversationalRetrievalChain.from_llm(llm = llm, retriever = vectorestore.as_retriever(),
                                            return_source_documents=True)

In [ ]:

class Chatbot:
    def __init__(self):
        self.splitter = RecursiveCharacterTextSplitter(chunk_size=1000 ,chunk_overlap=100)
        self.embeddings = OpenAIEmbeddings()
        self.llm = ChatOpenAI(model_name="gpt-3.5-turbo")

    def load_document(self, files):
        text=""
        for file in files:
            pdf = PdfReader(file)
            for page in pdf.pages:
                text += page.extract_text()
        return text
    
    def load_directory(self, directory):
        loader = PyPDFDirectoryLoader(directory)
        return loader.load()

    
    def split_text(self, text):
        return self.splitter.split_documents(text)


    def vectorize(self, chunks):
        vectorestore = Chroma.from_documents(chunks, self.embeddings)
        return vectorestore
    
    def build_chain(self, vectorstore):
        chain = ConversationalRetrievalChain.from_llm(llm = self.llm, retriever = vectorstore.as_retriever(),
                                            return_source_documents=True)
        return chain

    

In [15]:
chatbot = Chatbot()
if pdf:
    text = chatbot.load_document(pdf)
    chunks = chatbot.split_text(text)
    vectorstore = chatbot.vectorize(chunks)
    chain = chatbot.build_chain(vectorstore)

OSError: [Errno 22] Invalid argument: '\x07'

In [ ]:
import streamlit as st
import openai
from dotenv import load_dotenv
from Chatbot import Chatbot
import os

st.title("Furnace Whisper")

# load the API key from the .env file
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

with st.sidebar:

   # pdfs = st.file_uploader("Upload a PDF", type=["pdf"], accept_multiple_files=True)
    chatbot = Chatbot()
    
    docs = chatbot.load_directory("directory/")
    chunks = chatbot.split_text(docs)
    vectorstore = chatbot.vectorize(chunks)
    chain = chatbot.build_chain(vectorstore)
    
    st.write("Ready to chat!")
    st.write("Ask a question below and I'll try to answer it.")
    question = st.text_input("Question:")
    if question:
        chat_history = []

        response = chain({"question": question,  "chat_history": chat_history})
        chat_history.append((question, response["answer"]))
        st.write(response["answer"])
"""  
    if pdfs:
        text = chatbot.load_document(pdfs)
        chunks = chatbot.split_text(text)
        vectorstore = chatbot.vectorize(chunks)
        chain = chatbot.build_chain(vectorstore)
        st.write("Ready to chat!")
        st.write("Ask a question below and I'll try to answer it.")
        question = st.text_input("Question:")
        if question:
            chat_history = []

            response = chain({"question": question,  "chat_history": chat_history})
            chat_history.append((question, response["answer"]))
            st.write(response["answer"]) """